<a href="https://colab.research.google.com/github/ASK120305/CNS/blob/main/DM-megaupdateBI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#preprocessing
import io
import pandas as pd
import numpy as np

# This script is adapted for Google Colab.
# It opens a file dialog to upload a CSV from the user's local machine,
# then runs basic preprocessing and (optionally) saves the cleaned CSV.

def upload_csv_from_colab():
    try:
        from google.colab import files
    except Exception as e:
        raise RuntimeError(
            "This helper requires Google Colab. Run this script in a Colab notebook."
        ) from e

    print("Please use the file chooser to upload a CSV file from your local device.")
    uploaded = files.upload()  # opens dialog

    if not uploaded:
        raise RuntimeError("No file uploaded.")

    # Take the first uploaded file
    filename = next(iter(uploaded))
    file_bytes = uploaded[filename]

    # Try reading the CSV (auto-detect encoding if possible)
    try:
        df = pd.read_csv(io.BytesIO(file_bytes))
    except Exception:
        # fallback with latin1 encoding
        df = pd.read_csv(io.BytesIO(file_bytes), encoding="latin1")

    return df, filename


def basic_cleaning(df):
    # Display initial info
    print("Original data shape:", df.shape)
    try:
        print(df.info())
    except Exception:
        pass
    print(df.head())

    # 1. Drop duplicate rows
    df = df.drop_duplicates()

    # 2. Handle missing values
    # Drop rows with all fields missing
    df = df.dropna(how="all")

    # For columns with too many missing values, drop column (example threshold 70% missing)
    missing_pct = df.isnull().mean()
    cols_to_drop = missing_pct[missing_pct > 0.7].index
    if len(cols_to_drop) > 0:
        print("Dropping columns with >70% missing values:", list(cols_to_drop))
    df = df.drop(columns=cols_to_drop)

    # For columns with moderate missing, fill with median (numeric) or mode (categorical)
    for col in df.columns:
        # numeric detection
        if pd.api.types.is_numeric_dtype(df[col]):
            if df[col].isnull().all():
                # leave column as-is (all NaN)
                continue
            df[col] = df[col].fillna(df[col].median())
        else:
            # for non-numeric, fill with mode if available, else with empty string
            try:
                mode_vals = df[col].mode()
                if not mode_vals.empty:
                    fill_val = mode_vals[0]
                else:
                    fill_val = ""
            except Exception:
                fill_val = ""
            df[col] = df[col].fillna(fill_val)

    # 3. General type conversion (example: convert 'date' columns to datetime)
    for col in df.columns:
        if "date" in col.lower():
            # coerce errors -> NaT for unparsable entries
            df[col] = pd.to_datetime(df[col], errors="coerce")

    # 4. Lowercase string columns and strip whitespace
    for col in df.select_dtypes(include=["object", "string"]).columns:
        # convert to string type safely, skip NaNs
        df[col] = df[col].astype("string").str.lower().str.strip().replace("<nan>", pd.NA)

    # 5. (Optional) Remove outliers from numerical columns (using Z-score)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if numeric_cols:
        try:
            from scipy.stats import zscore

            for col in numeric_cols:
                # zscore returns nan for constant columns; fillna with 0 so they are kept
                col_values = df[col].copy()
                # we compute zscore on the column, ignoring NaNs
                zs = zscore(col_values.fillna(col_values.mean()))
                mask = (np.abs(zs) < 3) | (col_values.isnull())
                df = df[mask]
        except Exception:
            # fallback to manual z-score if scipy not available
            for col in numeric_cols:
                col_values = df[col]
                if col_values.std(ddof=0) == 0 or np.isnan(col_values.std(ddof=0)):
                    continue
                z = (col_values - col_values.mean()) / col_values.std(ddof=0)
                mask = (np.abs(z) < 3) | (col_values.isnull())
                df = df[mask]

    print("Data after cleaning:", df.shape)
    print(df.head())

    return df


if __name__ == "__main__":
    # Upload CSV from local device (Colab file dialog)
    df, original_filename = upload_csv_from_colab()

    # Run basic cleaning
    cleaned_df = basic_cleaning(df)

    # Optionally save cleaned file back to the user's machine
    try:
        from google.colab import files

        out_name = f"cleaned_{original_filename}"
        cleaned_df.to_csv(out_name, index=False)
        print(f"Cleaned CSV saved as {out_name}. It will be downloaded to your local machine.")
        files.download(out_name)
    except Exception:
        # If download fails (e.g., not running in Colab) just print instruction
        print("Finished cleaning. To save the dataframe, call cleaned_df.to_csv('cleaned.csv', index=False).")

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

sns.set(style="whitegrid")
plt.rcParams.update({'figure.max_open_warning': 0})

# ---------- Step 1: Upload and load ----------
print("Please upload the CSV file (e.g. 'data.csv' from the Kaggle breast cancer dataset).")
uploaded = files.upload()

if not uploaded:
    print("No file uploaded. Exiting.")
    sys.exit(0)

csv_path = next(iter(uploaded.keys()))
df = pd.read_csv(csv_path)
print(f"Loaded: {csv_path}\n")

# ---------- Step 2: Clean & canonicalize ----------
# Drop completely empty trailing column if present (Kaggle dataset has Unnamed: 32)
if 'Unnamed: 32' in df.columns:
    df = df.drop(columns=['Unnamed: 32'])

# Drop 'id' column - not useful for analysis
if 'id' in df.columns:
    df = df.drop(columns=['id'])

# Map diagnosis to Outcome for compatibility with older scripts (M -> 1 malignant, B -> 0 benign)
if 'diagnosis' in df.columns and 'Outcome' not in df.columns:
    df['Outcome'] = df['diagnosis'].map({'M': 1, 'B': 0})

# ---------- Step 3: Basic inspection ----------
print("First 5 rows:")
display(df.head())

print("\nDataFrame info:")
display(df.info())

print("\nMissing values per column:")
display(df.isnull().sum())

# ---------- Step 4: Target (diagnosis / Outcome) overview ----------
if 'diagnosis' in df.columns:
    print("\nDiagnosis value counts:")
    display(df['diagnosis'].value_counts())

if 'Outcome' in df.columns:
    print("\nOutcome value counts (mapped):")
    display(df['Outcome'].value_counts())

# ---------- Step 5: Statistical summary (with medians, IQR, and mode) ----------
num_cols = df.select_dtypes(include=np.number).columns.tolist()
desc = df[num_cols].describe().T
desc['median'] = df[num_cols].median()
desc['IQR'] = df[num_cols].quantile(0.75) - df[num_cols].quantile(0.25)
desc['mode'] = df[num_cols].mode().iloc[0]   # <-- Added line for mode
print("\nStatistical summary for numeric columns:")
display(desc[['count','mean','std','min','25%','median','75%','max','IQR','mode']])

# ---------- Step 6: Correlation analysis (full but uncluttered) ----------
corr = df[num_cols].corr()

# Mask upper triangle for clarity
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(14,12))
sns.heatmap(corr, mask=mask, cmap='vlag', center=0, annot=False, fmt='.2f',
            linewidths=0.5, cbar_kws={"shrink": .6})
plt.title("Correlation Heatmap (full numeric features, upper triangle masked)", fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# ---------- Step 7: Concise correlation heatmap for top features related to Outcome ----------
if 'Outcome' in corr.columns:
    # Identify features most correlated with Outcome (absolute correlation)
    corr_with_outcome = corr['Outcome'].abs().sort_values(ascending=False)
    corr_with_outcome = corr_with_outcome.drop('Outcome', errors='ignore')  # remove self
    top_n = 12  # show concise matrix for top N features
    top_features = corr_with_outcome.head(top_n).index.tolist()
    concise_cols = top_features + ['Outcome']
    concise_corr = df[concise_cols].corr()

    # Mask upper triangle for neatness
    mask2 = np.triu(np.ones_like(concise_corr, dtype=bool))

    plt.figure(figsize=(10,8))
    sns.heatmap(concise_corr, mask=mask2, cmap='vlag', center=0, annot=True, fmt='.2f',
                annot_kws={"size":8}, linewidths=0.5, cbar_kws={"shrink": .6})
    plt.title(f"Concise Correlation (top {top_n} features by |corr| with Outcome)", fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    print("\nTop features by absolute correlation with Outcome:")
    display(corr_with_outcome.head(top_n))
else:
    print("No 'Outcome' column present, skipping top-feature concise correlation heatmap.")

# ---------- Step 8: Distributions and boxplots for top features ----------
# Choose top ~6 features (or fewer if dataset has fewer)
if 'Outcome' in df.columns:
    top_pairplot_n = 6
    top_for_plots = corr_with_outcome.head(top_pairplot_n).index.tolist()
else:
    # fallback: choose first 6 numeric columns
    top_for_plots = num_cols[:6]

print(f"\nPlotting distributions and boxplots for: {top_for_plots}")

# Histograms / KDEs by diagnosis
for col in top_for_plots:
    plt.figure(figsize=(8,4))
    if 'diagnosis' in df.columns:
        sns.kdeplot(data=df, x=col, hue='diagnosis', fill=True, common_norm=False, alpha=0.4)
        plt.title(f"{col} distribution by diagnosis")
    elif 'Outcome' in df.columns:
        sns.kdeplot(data=df, x=col, hue='Outcome', fill=True, common_norm=False, alpha=0.4)
        plt.title(f"{col} distribution by Outcome")
    else:
        sns.histplot(df[col], kde=True)
        plt.title(f"{col} distribution")
    plt.tight_layout()
    plt.show()

# Boxplots grouped by diagnosis/outcome
group_col = 'diagnosis' if 'diagnosis' in df.columns else ('Outcome' if 'Outcome' in df.columns else None)
if group_col:
    for col in top_for_plots:
        plt.figure(figsize=(8,4))
        sns.boxplot(x=group_col, y=col, data=df)
        plt.title(f"{col} by {group_col}")
        plt.tight_layout()
        plt.show()
else:
    print("No group column (diagnosis/Outcome) found; skipping grouped boxplots.")

# ---------- Step 9: Pairplot for the top few features (keeps it small to avoid clutter) ----------
pairplot_features = top_for_plots[:6]  # at most 6
if len(pairplot_features) >= 2:
    try:
        hue_arg = 'diagnosis' if 'diagnosis' in df.columns else ('Outcome' if 'Outcome' in df.columns else None)
        sns.pairplot(df[pairplot_features + ([hue_arg] if hue_arg else [])], hue=hue_arg, palette='Set2', diag_kind='kde', corner=True)
        plt.suptitle("Pairplot (top features)", y=1.02)
        plt.show()
    except Exception as e:
        print("Pairplot failed (too many points or another issue):", e)
else:
    print("Not enough features for pairplot.")

# ---------- Step 10: Quick takeaways printed ----------

print("\nQuick takeaways:")
if 'Outcome' in df.columns:
    # Print top 6 positive/negative correlations with outcome
    signed_corr = df[num_cols].corr()['Outcome'].drop('Outcome').sort_values()
    print("- Features most negatively correlated with Outcome (benign-associated):")
    display(signed_corr.head(6))
    print("- Features most positively correlated with Outcome (malignant-associated):")
    display(signed_corr.tail(6))
else:
    print("- No 'Outcome' to compute feature associations with. Inspect diagnosis column instead.")

print("""
Notes & next steps:
- The full heatmap (masked) shows all numeric correlations but can be cluttered for 30+ features.
- The concise matrix (above) shows only the top features most correlated with Outcome and is annotated for readability.
- You can optionally run feature selection, PCA, or build classification models (LogisticRegression / RandomForest) using these top features.
""")


In [ ]:
# Decision Tree classifier on Kaggle Breast Cancer (Wisconsin) dataset
# Run this in Google Colab. Upload the CSV when prompted.

from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

sns.set(style="whitegrid")
plt.rcParams.update({'figure.max_open_warning': 0})

# -------------------
# Upload / load CSV
# -------------------
print("Please upload the Kaggle breast cancer CSV (e.g. 'data.csv').")
uploaded = files.upload()

if not uploaded:
    raise SystemExit("No file uploaded. Exiting.")

csv_path = next(iter(uploaded.keys()))
df = pd.read_csv(csv_path)
print(f"Loaded file: {csv_path}\n")

# -------------------
# Clean / canonicalize
# -------------------
# Drop trailing empty column if present
if 'Unnamed: 32' in df.columns:
    df = df.drop(columns=['Unnamed: 32'])

# Drop id column (not informative)
if 'id' in df.columns:
    df = df.drop(columns=['id'])

# Map diagnosis to Outcome: M -> 1 (malignant), B -> 0 (benign)
if 'diagnosis' in df.columns and 'Outcome' not in df.columns:
    df['Outcome'] = df['diagnosis'].map({'M': 1, 'B': 0})
    print("Mapped 'diagnosis' to 'Outcome' (M->1, B->0)")

# -------------------
# Feature selection
# -------------------
features = [
    'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
    'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean'
]

existing_features = [f for f in features if f in df.columns]
if len(existing_features) < 4:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [c for c in numeric_cols if c != 'Outcome']
    existing_features = numeric_cols[:8]
    print(f"Using fallback numeric features: {existing_features}")

features = existing_features

# Ensure Outcome exists
if 'Outcome' not in df.columns:
    raise SystemExit("No 'Outcome' or 'diagnosis' column found in the uploaded file. Exiting.")

# Drop rows with missing values in features/target
df_model = df.dropna(subset=features + ['Outcome']).copy()

# Prepare X, y
y = df_model['Outcome'].astype(int)
X = df_model[features]

# -------------------
# Train/Test split (deterministic, stratified)
# -------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# -------------------
# Decision Tree classifier
# -------------------
dt_classifier = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_classifier.fit(X_train, y_train)

y_pred_dt = dt_classifier.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"\nAccuracy of Decision Tree (max_depth=3): {accuracy_dt:.4f}\n")
print("Classification report - Decision Tree:")
print(classification_report(y_test, y_pred_dt, labels=[0,1], target_names=['Benign','Malignant'], zero_division=0))

# Plot Decision Tree
plt.figure(figsize=(16,9))
plot_tree(dt_classifier, filled=True, feature_names=features, class_names=['Benign','Malignant'], fontsize=10)
plt.title("Decision Tree (max_depth=3) - Breast Cancer")
plt.show()

# Feature importances
importances = dt_classifier.feature_importances_
if np.any(importances):
    feat_imp = pd.Series(importances, index=features).sort_values(ascending=True)
    plt.figure(figsize=(8,5))
    feat_imp.plot(kind='barh', color='C1')
    plt.title("Decision Tree Feature Importances")
    plt.xlabel("Importance")
    plt.tight_layout()
    plt.show()

# Confusion matrix - Decision Tree
cm_dt = confusion_matrix(y_test, y_pred_dt, labels=[0,1])
plt.figure(figsize=(6,5))
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues', xticklabels=['Benign','Malignant'], yticklabels=['Benign','Malignant'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Decision Tree')
plt.show()

print("Done. Decision Tree model trained and evaluated on features:", features)

In [ ]:
# Gaussian Naive Bayes classifier on Kaggle Breast Cancer (Wisconsin) dataset
# Run this in Google Colab. Upload the same CSV when prompted.

from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

sns.set(style="whitegrid")
plt.rcParams.update({'figure.max_open_warning': 0})

# -------------------
# Upload / load CSV
# -------------------
print("Please upload the Kaggle breast cancer CSV (e.g. 'data.csv').")
uploaded = files.upload()

if not uploaded:
    raise SystemExit("No file uploaded. Exiting.")

csv_path = next(iter(uploaded.keys()))
df = pd.read_csv(csv_path)
print(f"Loaded file: {csv_path}\n")

# -------------------
# Clean / canonicalize
# -------------------
# Drop trailing empty column if present
if 'Unnamed: 32' in df.columns:
    df = df.drop(columns=['Unnamed: 32'])

# Drop id column (not informative)
if 'id' in df.columns:
    df = df.drop(columns=['id'])

# Map diagnosis to Outcome: M -> 1 (malignant), B -> 0 (benign)
if 'diagnosis' in df.columns and 'Outcome' not in df.columns:
    df['Outcome'] = df['diagnosis'].map({'M': 1, 'B': 0})
    print("Mapped 'diagnosis' to 'Outcome' (M->1, B->0)")

# -------------------
# Feature selection (must match the Decision Tree file for comparability)
# -------------------
features = [
    'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
    'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean'
]

existing_features = [f for f in features if f in df.columns]
if len(existing_features) < 4:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [c for c in numeric_cols if c != 'Outcome']
    existing_features = numeric_cols[:8]
    print(f"Using fallback numeric features: {existing_features}")

features = existing_features

# Ensure Outcome exists
if 'Outcome' not in df.columns:
    raise SystemExit("No 'Outcome' or 'diagnosis' column found in the uploaded file. Exiting.")

# Drop rows with missing values in features/target
df_model = df.dropna(subset=features + ['Outcome']).copy()

# Prepare X, y
y = df_model['Outcome'].astype(int)
X = df_model[features]

# -------------------
# Train/Test split (must use same deterministic params as Decision Tree script)
# -------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# -------------------
# Gaussian Naive Bayes classifier
# -------------------
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
y_pred_nb = nb_classifier.predict(X_test)

accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"\nAccuracy of Gaussian Naive Bayes: {accuracy_nb:.4f}\n")
print("Classification report - Naive Bayes:")
print(classification_report(y_test, y_pred_nb, labels=[0,1], target_names=['Benign','Malignant'], zero_division=0))

# Confusion matrix - Naive Bayes
cm_nb = confusion_matrix(y_test, y_pred_nb, labels=[0,1])
plt.figure(figsize=(6,5))
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Blues', xticklabels=['Benign','Malignant'], yticklabels=['Benign','Malignant'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Naive Bayes')
plt.show()

# Optional: probability distributions for the positive class (Malignant)
if hasattr(nb_classifier, "predict_proba"):
    y_prob = nb_classifier.predict_proba(X_test)[:, 1]
    plt.figure(figsize=(8,4))
    sns.histplot(y_prob, bins=20, kde=True, color='C2')
    plt.title("Predicted probability distribution (Malignant) - Naive Bayes")
    plt.xlabel("Predicted probability")
    plt.tight_layout()
    plt.show()

print("Done. GaussianNB model trained and evaluated on features:", features)

In [ ]:
# Agglomerative Clustering (Colab-ready) for uploaded dataset (Breast Cancer / Diabetes / general CSV)
# - Upload your CSV when prompted in Colab (e.g. the Kaggle breast-cancer-wisconsin-data data.csv).
# - The script will canonicalize the breast-cancer dataset if detected (drop id/Unnamed: 32, map diagnosis -> Outcome).
# - For diabetes-like datasets it will also optionally replace medically-invalid zeros (Glucose, BloodPressure, SkinThickness, Insulin, BMI).
# - Produces a dendrogram (sampled), a cluster scatter using PCA (2 components), cluster statistics, silhouette score,
#   and saves a clustered CSV that you can download from Colab.
#
# Run this as a single cell in Google Colab.

from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import io
import sys
import random
import warnings
warnings.filterwarnings('ignore')

sns.set(style="whitegrid")
plt.rcParams.update({'figure.max_open_warning': 0})

# -------------------------
# 1) Upload & load CSV
# -------------------------
print("Please upload your CSV file (e.g. Kaggle breast-cancer data.csv).")
uploaded = files.upload()  # opens Colab file picker

if not uploaded:
    print("No file uploaded. Exiting.")
    raise SystemExit("No file uploaded")

csv_name = next(iter(uploaded.keys()))
print(f"Loaded file: {csv_name}")
# Read into DataFrame
try:
    df = pd.read_csv(io.BytesIO(uploaded[csv_name]))
except Exception as e:
    print("Failed to read uploaded CSV:", e)
    raise

print(f"Initial shape: {df.shape}")
print("Columns detected:", df.columns.tolist())

# -------------------------
# 2) Dataset-specific cleaning
# -------------------------
# Handle common Kaggle breast-cancer dataset quirks
if 'Unnamed: 32' in df.columns:
    df = df.drop(columns=['Unnamed: 32'])
    print("Dropped column: Unnamed: 32")

if 'id' in df.columns:
    df = df.drop(columns=['id'])
    print("Dropped column: id")

# Map diagnosis -> Outcome for compatibility
if 'diagnosis' in df.columns and 'Outcome' not in df.columns:
    df['Outcome'] = df['diagnosis'].map({'M': 1, 'B': 0})
    print("Mapped 'diagnosis' to 'Outcome' (M->1, B->0)")

# If this is the diabetes dataset (or similar) handle medically-invalid zeros
zero_not_allowed = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
zero_cols_present = [c for c in zero_not_allowed if c in df.columns]
if zero_cols_present:
    print("Replacing medically-invalid zeros with column means for:", zero_cols_present)
    for col in zero_cols_present:
        df[col] = df[col].replace(0, np.nan)
        df[col].fillna(df[col].mean(), inplace=True)

# -------------------------
# 3) Feature selection
# -------------------------
# Use numeric columns only (exclude Outcome if present)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Outcome' in numeric_cols:
    numeric_cols.remove('Outcome')

if len(numeric_cols) < 2:
    raise SystemExit("Need at least two numeric features for clustering. Found: " + str(numeric_cols))

print(f"Numeric features used for clustering ({len(numeric_cols)}): {numeric_cols}")

X = df[numeric_cols].copy()

# Optionally drop columns with very low variance (near-constant) to reduce noise
var_threshold = 1e-6
low_var = X.var().loc[lambda s: s <= var_threshold].index.tolist()
if low_var:
    X = X.drop(columns=low_var)
    print("Dropped near-constant columns:", low_var)

# -------------------------
# 4) Scale features
# -------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Data scaled. Shape:", X_scaled.shape)

# -------------------------
# 5) Dendrogram (sampled for clarity)
# -------------------------
sample_size = min(60, X_scaled.shape[0])  # keep sample small for clear dendrogram
sample_indices = np.random.choice(X_scaled.shape[0], sample_size, replace=False)
X_sample = X_scaled[sample_indices]

print(f"\nGenerating dendrogram using a sample of {sample_size} rows...")
plt.figure(figsize=(12, 6))
try:
    linkage_matrix = linkage(X_sample, method='ward')
    dendrogram(linkage_matrix, truncate_mode='level', p=5, leaf_rotation=90, leaf_font_size=8)
    plt.title("Dendrogram (sampled) - Ward linkage", fontsize=14, fontweight='bold')
    plt.xlabel("Sample index (truncated)")
    plt.ylabel("Distance")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Dendrogram generation failed:", e)

# -------------------------
# 6) Agglomerative clustering
# -------------------------
n_clusters = 3  # you can change this
print(f"\nApplying AgglomerativeClustering with n_clusters={n_clusters} (ward linkage).")
agg = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
labels = agg.fit_predict(X_scaled)
df['Cluster'] = labels

# -------------------------
# 7) Visualization: PCA scatter for first two components (clear & concise)
# -------------------------
print("Creating 2D PCA scatter plot of clusters (for visualization).")
pca = PCA(n_components=2, random_state=42)
pc = pca.fit_transform(X_scaled)

plt.figure(figsize=(10, 7))
palette = sns.color_palette("tab10", n_colors=max(3, n_clusters))
for i in range(n_clusters):
    mask = labels == i
    plt.scatter(pc[mask, 0], pc[mask, 1], s=60, alpha=0.75, label=f"Cluster {i}", edgecolor='k')
plt.xlabel("PC1 (%.1f%% var)" % (pca.explained_variance_ratio_[0]*100))
plt.ylabel("PC2 (%.1f%% var)" % (pca.explained_variance_ratio_[1]*100))
plt.title(f"Agglomerative Clustering (n={n_clusters}) projected to 2 PC", fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# -------------------------
# 8) Cluster analysis & silhouette
# -------------------------
print("\n=== Cluster distribution ===")
cluster_counts = df['Cluster'].value_counts().sort_index()
print(cluster_counts.to_string())

print("\n=== Cluster means (selected features) ===")
for i in range(n_clusters):
    subset = df[df['Cluster'] == i][numeric_cols]
    if len(subset) > 0:
        print(f"\nCluster {i} (n={len(subset)}):")
        display(subset.mean().round(3))

# Silhouette score (only if more than 1 cluster and fewer clusters than samples)
silhouette_avg = None
try:
    if len(set(labels)) > 1 and X_scaled.shape[0] > len(set(labels)):
        silhouette_avg = silhouette_score(X_scaled, labels)
        print(f"\nSilhouette Score: {silhouette_avg:.4f}")
    else:
        print("\nSilhouette Score not computed (need >1 cluster and more samples than clusters).")
except Exception as e:
    print("Silhouette score calculation error:", e)

# -------------------------
# 9) Save clustered dataset and provide download link
# -------------------------
out_name = f"clustered_{csv_name}"
try:
    df.to_csv(out_name, index=False)
    print(f"\nClustered dataset saved as: {out_name}")
    # Make file available for download in Colab
    files.download(out_name)
except Exception as e:
    print("Could not save or download clustered CSV:", e)

# -------------------------
# 10) If Outcome present: cross-tab
# -------------------------
if 'Outcome' in df.columns or 'diagnosis' in df.columns:
    # Prefer numeric Outcome; if not present, use diagnosis
    group_col = 'Outcome' if 'Outcome' in df.columns else 'diagnosis'
    try:
        print(f"\nCluster vs {group_col} cross-tab:")
        ct = pd.crosstab(df['Cluster'], df[group_col], margins=True)
        display(ct)
    except Exception as e:
        print("Error computing cluster vs outcome cross-tab:", e)

print("\nClustering complete. Adjust `n_clusters` or feature set to explore other clusterings.")

In [ ]:
# DBSCAN clustering (Colab-ready) for uploaded dataset (Breast Cancer / Diabetes / general CSV)
# - Paste this cell into a Google Colab notebook and run.
# - Upload your CSV when prompted (e.g. Kaggle breast-cancer-wisconsin-data data.csv).
# - The script canonicalizes the breast-cancer CSV (drops id/Unnamed: 32, maps diagnosis->Outcome),
#   optionally replaces medically-invalid zeros for diabetes-like datasets, runs DBSCAN on scaled numeric features,
#   visualizes clusters on a 2D PCA projection, computes silhouette score (excluding noise), and downloads the clustered CSV.
#
# Usage: run the cell, upload the CSV, and inspect the printed outputs & plots.
from google.colab import files
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

sns.set(style="whitegrid")
plt.rcParams.update({'figure.max_open_warning': 0})

# -------------------------
# User-tunable DBSCAN params
# -------------------------
# You can change these before running the cell again.
EPS = 0.5
MIN_SAMPLES = 5
N_PCA_COMPONENTS = 2
RANDOM_STATE = 42

# -------------------------
# 1) Upload & load CSV
# -------------------------
print("Please upload your CSV file (e.g. Kaggle breast-cancer data.csv).")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded. Exiting.")

csv_name = next(iter(uploaded.keys()))
print(f"Loaded file: {csv_name}")
try:
    df = pd.read_csv(io.BytesIO(uploaded[csv_name]))
except Exception as e:
    raise SystemExit(f"Failed to read CSV: {e}")

print(f"Initial shape: {df.shape}")
print("Columns:", df.columns.tolist())

# -------------------------
# 2) Dataset-specific cleaning
# -------------------------
# Common Kaggle breast-cancer file quirks
if 'Unnamed: 32' in df.columns:
    df = df.drop(columns=['Unnamed: 32'])
    print("Dropped column: Unnamed: 32")

if 'id' in df.columns:
    df = df.drop(columns=['id'])
    print("Dropped column: id")

# Map diagnosis -> Outcome for compatibility
if 'diagnosis' in df.columns and 'Outcome' not in df.columns:
    df['Outcome'] = df['diagnosis'].map({'M': 1, 'B': 0})
    print("Mapped 'diagnosis' to 'Outcome' (M->1, B->0)")

# Replace medically-invalid zeros if diabetes-like columns exist
zero_not_allowed = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
zero_cols_present = [c for c in zero_not_allowed if c in df.columns]
if zero_cols_present:
    print("Replacing medically-invalid zeros with column means for:", zero_cols_present)
    for c in zero_cols_present:
        df[c] = df[c].replace(0, np.nan)
        df[c].fillna(df[c].mean(), inplace=True)

# -------------------------
# 3) Feature selection (numeric features)
# -------------------------
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Outcome' in numeric_cols:
    numeric_cols.remove('Outcome')

if len(numeric_cols) < 2:
    raise SystemExit(f"Need at least 2 numeric features for clustering. Found: {numeric_cols}")

print(f"Numeric features used for clustering ({len(numeric_cols)}): {numeric_cols}")

X = df[numeric_cols].copy()

# Optionally drop near-constant columns
var_threshold = 1e-8
low_var_cols = X.var().loc[lambda s: s <= var_threshold].index.tolist()
if low_var_cols:
    print("Dropping near-constant columns:", low_var_cols)
    X = X.drop(columns=low_var_cols)
    numeric_cols = [c for c in numeric_cols if c not in low_var_cols]

# -------------------------
# 4) Scale features
# -------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Data scaled. Shape:", X_scaled.shape)

# -------------------------
# 5) PCA for 2D visualization
# -------------------------
pca = PCA(n_components=N_PCA_COMPONENTS, random_state=RANDOM_STATE)
pc = pca.fit_transform(X_scaled)
explained = pca.explained_variance_ratio_

# -------------------------
# 6) Run DBSCAN
# -------------------------
print(f"\nRunning DBSCAN with eps={EPS}, min_samples={MIN_SAMPLES} ...")
db = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES)
labels = db.fit_predict(X_scaled)
df['Cluster'] = labels

unique_labels = set(labels)
n_clusters = len(unique_labels) - (1 if -1 in labels else 0)
n_noise = int((labels == -1).sum())

print(f"Clusters found (excluding noise): {n_clusters}")
print(f"Noise points: {n_noise}")

# -------------------------
# 7) Visualization (PCA scatter)
# -------------------------
plt.figure(figsize=(10, 7))
palette = sns.color_palette("tab10", n_colors=max(3, len(unique_labels)))
for lbl in sorted(unique_labels):
    mask = labels == lbl
    if lbl == -1:
        # noise as gray x
        plt.scatter(pc[mask, 0], pc[mask, 1], c='lightgray', marker='x', s=50, alpha=0.6, label='Noise')
    else:
        color = palette[lbl % len(palette)]
        plt.scatter(pc[mask, 0], pc[mask, 1], c=[color], s=60, alpha=0.8, edgecolor='k', label=f'Cluster {lbl}')

plt.xlabel(f"PC1 ({explained[0]*100:.1f}% var)")
plt.ylabel(f"PC2 ({explained[1]*100:.1f}% var)" if len(explained) > 1 else "PC2")
plt.title(f"DBSCAN Clustering (eps={EPS}, min_samples={MIN_SAMPLES}) projected to 2 PC", fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# -------------------------
# 8) Cluster counts & summary stats
# -------------------------
print("\n=== Cluster distribution ===")
cluster_counts = df['Cluster'].value_counts().sort_index()
print(cluster_counts.to_string())

print("\n=== Cluster means (numeric features) ===")
for lbl in sorted(unique_labels):
    subset = df[df['Cluster'] == lbl][numeric_cols]
    if len(subset) > 0:
        print(f"\nCluster {lbl} (n={len(subset)}):")
        display(subset.mean().round(3))

# -------------------------
# 9) Silhouette score (exclude noise)
# -------------------------
silhouette_avg = None
try:
    if n_clusters > 1:
        mask_non_noise = labels != -1
        if mask_non_noise.sum() > 1:
            silhouette_avg = silhouette_score(X_scaled[mask_non_noise], labels[mask_non_noise])
            print(f"\nSilhouette Score (excluding noise): {silhouette_avg:.4f}")
        else:
            print("\nNot enough non-noise points for silhouette score")
    else:
        print("\nNot enough clusters for silhouette score (need >1 cluster)")
except Exception as e:
    print(f"\nSilhouette score error: {e}")

# -------------------------
# 10) Save clustered CSV and provide download
# -------------------------
out_name = f"clustered_{csv_name}"
try:
    df.to_csv(out_name, index=False)
    print(f"\nClustered dataset saved as: {out_name}")
    files.download(out_name)
except Exception as e:
    print("Could not save or download clustered CSV:", e)

# -------------------------
# 11) If Outcome present: cross-tab
# -------------------------
if 'Outcome' in df.columns or 'diagnosis' in df.columns:
    group_col = 'Outcome' if 'Outcome' in df.columns else 'diagnosis'
    try:
        print(f"\nCluster vs {group_col} cross-tab:")
        ct = pd.crosstab(df['Cluster'], df[group_col], margins=True)
        display(ct)
    except Exception as e:
        print("Error computing cross-tab:", e)

# -------------------------
# 12) Parameter tuning suggestions
# -------------------------
print("\n=== Parameter tuning suggestions ===")
print(f"Current parameters: eps={EPS}, min_samples={MIN_SAMPLES}")
print(f"Results: {n_clusters} clusters, {n_noise} noise points")
if n_clusters == 0:
    print("Suggestion: Try decreasing eps to allow DBSCAN to discover tighter clusters, or increase min_samples if too many small clusters.")
elif n_noise > len(X_scaled) * 0.1:
    print("Suggestion: Consider increasing eps or decreasing min_samples to reduce noise points.")
elif n_clusters == 1:
    print("Suggestion: Decrease eps to attempt to split the data into multiple clusters.")
else:
    print("Suggestion: Try varying eps in a small range around the current value and tune min_samples to balance cluster count vs noise.")

print("\nDBSCAN clustering complete.")

In [ ]:
# K-Means Clustering (Colab-ready) for uploaded dataset (Breast Cancer / Diabetes / general CSV)
# - Run this cell in Google Colab. Upload your CSV when prompted (e.g. Kaggle breast-cancer data.csv).
# - The script canonicalizes common breast-cancer CSV quirks (drops id/Unnamed: 32, maps diagnosis -> Outcome),
#   optionally replaces medically-invalid zeros for diabetes-like datasets, runs KMeans, shows the Elbow plot,
#   visualizes clusters in 2D using PCA, computes silhouette score, and downloads the clustered CSV.
#
# Usage: run the cell, upload your CSV, adjust CHOSEN_K if you want a different number of clusters.

from google.colab import files
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")
plt.rcParams.update({'figure.max_open_warning': 0})

# -----------------------
# User-tunable parameters
# -----------------------
K_RANGE = range(1, 11)    # range to evaluate for elbow method
CHOSEN_K = 3              # default k to fit after elbow (change if desired)
N_INIT = 10               # KMeans n_init
RANDOM_STATE = 42

# -----------------------
# 1) Upload & load CSV
# -----------------------
print("Please upload your CSV file (e.g. Kaggle breast-cancer data.csv).")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded. Exiting.")

csv_name = next(iter(uploaded.keys()))
print(f"Loaded file: {csv_name}")
try:
    df = pd.read_csv(io.BytesIO(uploaded[csv_name]))
except Exception as e:
    raise SystemExit(f"Failed to read CSV: {e}")

print(f"Initial shape: {df.shape}")
print("Columns:", df.columns.tolist())

# -----------------------
# 2) Canonicalize dataset
# -----------------------
# Drop common Kaggle breast-cancer quirks
if 'Unnamed: 32' in df.columns:
    df = df.drop(columns=['Unnamed: 32'])
    print("Dropped column: Unnamed: 32")

if 'id' in df.columns:
    df = df.drop(columns=['id'])
    print("Dropped column: id")

# Map diagnosis -> Outcome for compatibility
if 'diagnosis' in df.columns and 'Outcome' not in df.columns:
    df['Outcome'] = df['diagnosis'].map({'M': 1, 'B': 0})
    print("Mapped 'diagnosis' to numeric 'Outcome' (M->1, B->0)")

# Replace medically-invalid zeros if diabetes-like columns exist
zero_not_allowed = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
zero_cols_present = [c for c in zero_not_allowed if c in df.columns]
if zero_cols_present:
    print("Replacing medically-invalid zeros with column means for:", zero_cols_present)
    for c in zero_cols_present:
        df[c] = df[c].replace(0, np.nan)
        df[c].fillna(df[c].mean(), inplace=True)

# -----------------------
# 3) Feature selection
# -----------------------
# Use numeric features only for clustering; exclude Outcome if present
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Outcome' in numeric_cols:
    numeric_cols.remove('Outcome')

if len(numeric_cols) < 2:
    raise SystemExit("Need at least two numeric features for clustering. Found: " + str(numeric_cols))

print(f"Numeric features used for clustering ({len(numeric_cols)}): {numeric_cols}")

X = df[numeric_cols].copy()

# Optionally drop near-constant columns
low_var_thresh = 1e-8
low_var_cols = X.var().loc[lambda s: s <= low_var_thresh].index.tolist()
if low_var_cols:
    print("Dropping near-constant columns:", low_var_cols)
    X = X.drop(columns=low_var_cols)
    numeric_cols = [c for c in numeric_cols if c not in low_var_cols]

# -----------------------
# 4) Scale features
# -----------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Data scaled. Shape:", X_scaled.shape)

# -----------------------
# 5) Elbow method (WCSS) to suggest k
# -----------------------
wcss = []
print("\nComputing WCSS for K in", list(K_RANGE))
for k in K_RANGE:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=N_INIT)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

# -----------------------
# 6) PCA for visualization
# -----------------------
pca = PCA(n_components=2, random_state=RANDOM_STATE)
pc = pca.fit_transform(X_scaled)
explained = pca.explained_variance_ratio_

# -----------------------
# 7) Fit KMeans with chosen k and get labels
# -----------------------
chosen_k = CHOSEN_K
if chosen_k < 1 or chosen_k > 50:
    chosen_k = 3
    print(f"Invalid CHOSEN_K, defaulting to {chosen_k}")

kmeans_final = KMeans(n_clusters=chosen_k, random_state=RANDOM_STATE, n_init=N_INIT)
labels = kmeans_final.fit_predict(X_scaled)
df['Cluster'] = labels

# Project centroids into PCA space for plotting
centroids_scaled = kmeans_final.cluster_centers_
centroids_pca = pca.transform(centroids_scaled)

# -----------------------
# 8) Plot Elbow (left) and PCA cluster scatter (right)
# -----------------------
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Elbow plot
axes[0].plot(list(K_RANGE), wcss, 'o-b', linewidth=2, markersize=6)
axes[0].set_xlabel("Number of clusters (k)")
axes[0].set_ylabel("WCSS (Within-Cluster Sum of Squares)")
axes[0].set_title("Elbow Method for Optimal k")
axes[0].grid(alpha=0.3)
axes[0].axvline(chosen_k, color='gray', linestyle='--', label=f'chosen k={chosen_k}')
axes[0].legend()

# PCA scatter with clusters
palette = sns.color_palette("tab10", n_colors=max(3, chosen_k))
for lbl in sorted(set(labels)):
    mask = labels == lbl
    axes[1].scatter(pc[mask, 0], pc[mask, 1], s=60, alpha=0.75, edgecolor='k', label=f'Cluster {lbl}')
# plot centroids
axes[1].scatter(centroids_pca[:, 0], centroids_pca[:, 1], c='black', marker='x', s=200, linewidths=3, label='Centroids')
axes[1].set_xlabel(f"PC1 ({explained[0]*100:.1f}% var)")
axes[1].set_ylabel(f"PC2 ({explained[1]*100:.1f}% var)")
axes[1].set_title(f"K-Means Clustering (k={chosen_k}) projected to 2 PC")
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# -----------------------
# 9) Cluster evaluation & summaries
# -----------------------
n_clusters_found = len(set(labels)) - (1 if -1 in labels else 0)
print(f"\nClusters found: {n_clusters_found} (k requested: {chosen_k})")
print("Cluster distribution:")
print(df['Cluster'].value_counts().sort_index().to_string())

print("\nCluster means (original feature space):")
for lbl in sorted(set(labels)):
    subset = df[df['Cluster'] == lbl][numeric_cols]
    if len(subset) > 0:
        print(f"\nCluster {lbl} (n={len(subset)}):")
        display(subset.mean().round(3))

# Silhouette score (only valid if more than 1 cluster)
try:
    if len(set(labels)) > 1:
        sil = silhouette_score(X_scaled, labels)
        print(f"\nSilhouette Score: {sil:.4f}")
    else:
        print("\nSilhouette Score: not applicable (only one cluster present).")
except Exception as e:
    print("\nSilhouette score calculation error:", e)

# -----------------------
# 10) Save clustered CSV and provide download
# -----------------------
out_name = f"clustered_{csv_name}"
try:
    df.to_csv(out_name, index=False)
    print(f"\nClustered dataset saved as: {out_name}")
    files.download(out_name)
except Exception as e:
    print("Could not save or download clustered CSV:", e)

# -----------------------
# 11) If Outcome present: cross-tab
# -----------------------
if 'Outcome' in df.columns or 'diagnosis' in df.columns:
    group_col = 'Outcome' if 'Outcome' in df.columns else 'diagnosis'
    try:
        print(f"\nCluster vs {group_col} cross-tab:")
        ct = pd.crosstab(df['Cluster'], df[group_col], margins=True)
        display(ct)
    except Exception as e:
        print("Error computing cluster vs outcome cross-tab:", e)

print("\nK-Means clustering complete. Adjust CHOSEN_K or K_RANGE and re-run to explore other clusterings.")

In [ ]:
# Apriori / FP-Growth (Colab-ready, fast & safe)
# Paste this whole file into a Colab cell and run. It will prompt you to upload a CSV.
# Designed to be equivalent to the "apriori_vscode_fast.py" desktop script but adapted
# for Google Colab: uses files.upload(), files.download(), and installs mlxtend if needed.
#
# Key behaviours:
#  - Auto-detects transactional datasets (Groceries / Online Retail) or converts tabular data
#    (e.g., breast-cancer) into transactions by binning numeric columns.
#  - Converts one-hot matrix to boolean dtype to satisfy mlxtend and speed up mining.
#  - Prunes low-support item columns and optionally auto-reduces top-K items to avoid combinatorial explosion.
#  - Uses fpgrowth by default (faster than apriori) and limits max itemset length for safety.
#  - Saves frequent itemsets and association rules CSVs and downloads them automatically.
#
# Tweak PARAMETERS below before running if desired.

# Install mlxtend if not installed (works in Colab)
try:
    import mlxtend
except Exception:
    import sys, subprocess
    print("Installing mlxtend...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "mlxtend"])
    print("mlxtend installed.")

from google.colab import files
import io
import math
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display

warnings.filterwarnings("ignore")
sns.set(style="whitegrid")
plt.rcParams.update({'figure.max_open_warning': 0})

from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

# ---------------------- PARAMETERS ----------------------
MIN_SUPPORT = 0.12       # relative support threshold (0..1)
MIN_CONFIDENCE = 0.6    # minimum confidence for rules
MIN_LIFT = 1.2          # minimum lift for "strong" rules
MAX_LEN = 3             # maximum itemset length (reduce complexity)
ALGO = "fpgrowth"       # "fpgrowth" (recommended) or "apriori"
TOP_N = 20              # how many top itemsets/rules to display
AUTO_REDUCE = True      # if True and many items, keep only TOP_K_ITEMS by support
TOP_K_ITEMS = 100       # used when AUTO_REDUCE=True
# ------------------------------------------------------

print("Please upload your CSV file (e.g. Kaggle breast-cancer data.csv or diabetes.csv).")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded. Exiting.")

fname = next(iter(uploaded.keys()))
print(f"Loaded: {fname}")

# Read CSV into DataFrame
try:
    df = pd.read_csv(io.BytesIO(uploaded[fname]))
except Exception as e:
    raise SystemExit(f"Failed to read uploaded CSV: {e}")

print("Initial shape:", df.shape)
print("Columns:", df.columns.tolist())

# Canonicalize common Kaggle breast-cancer quirks
if 'Unnamed: 32' in df.columns:
    df = df.drop(columns=['Unnamed: 32'])
    print("Dropped column: Unnamed: 32")
if 'id' in df.columns:
    df = df.drop(columns=['id'])
    print("Dropped column: id")
if 'diagnosis' in df.columns and 'Outcome' not in df.columns:
    df['Outcome'] = df['diagnosis'].map({'M': 1, 'B': 0})
    print("Mapped 'diagnosis' -> 'Outcome' (M->1, B->0)")

# Replace medically-invalid zeros for diabetes-like columns (if present)
zero_not_allowed = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
zero_cols_present = [c for c in zero_not_allowed if c in df.columns]
if zero_cols_present:
    print("Replacing medically-invalid zeros with column means for:", zero_cols_present)
    for c in zero_cols_present:
        df[c] = df[c].replace(0, np.nan)
        df[c].fillna(df[c].mean(), inplace=True)

# Detect transactional dataset formats
is_groceries = {'Member_number', 'Date', 'ItemDescription'}.issubset(set(df.columns))
is_online_retail = {'InvoiceNo', 'Description'}.issubset(set(df.columns))

# Build one-hot transaction matrix
if is_groceries:
    print("Detected Groceries-style transactional dataset.")
    TRANSACTION_ID_COLUMNS = ['Member_number', 'Date']
    ITEM_COLUMN = 'ItemDescription'
    df = df.drop_duplicates(subset=TRANSACTION_ID_COLUMNS + [ITEM_COLUMN], keep='first')
    basket_sets = (df.groupby(TRANSACTION_ID_COLUMNS)[ITEM_COLUMN]
                  .apply(lambda x: pd.Series(1, index=x))
                  .unstack(fill_value=0))
    basket_sets.columns.name = None
    basket_sets.index.names = TRANSACTION_ID_COLUMNS
    transactions_ohe = basket_sets.applymap(lambda x: 1 if x > 0 else 0)

elif is_online_retail:
    print("Detected Online Retail transactional dataset.")
    TRANSACTION_ID_COLUMNS = ['InvoiceNo']
    ITEM_COLUMN = 'Description'
    df = df.drop_duplicates(subset=TRANSACTION_ID_COLUMNS + [ITEM_COLUMN], keep='first')
    basket_sets = (df.groupby(TRANSACTION_ID_COLUMNS)[ITEM_COLUMN]
                  .apply(lambda x: pd.Series(1, index=x))
                  .unstack(fill_value=0))
    basket_sets.columns.name = None
    basket_sets.index.names = TRANSACTION_ID_COLUMNS
    transactions_ohe = basket_sets.applymap(lambda x: 1 if x > 0 else 0)

else:
    # Tabular dataset (e.g., breast-cancer) -> convert rows to transactions by binning numeric columns
    print("Non-transactional/tabular dataset detected. Binning numeric columns into Low/Med/High and using categorical columns.")
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'Outcome' in numeric_cols:
        numeric_cols.remove('Outcome')
    if len(numeric_cols) == 0:
        raise SystemExit("No numeric columns to bin; cannot create transactions for tabular dataset.")

    binned_cols = []
    binary_cols = []
    for col in numeric_cols:
        try:
            if df[col].nunique(dropna=True) <= 2:
                df[col] = df[col].astype(str)
                binary_cols.append(col)
            else:
                df[f"{col}_bin"] = pd.qcut(df[col].rank(method="first"), q=3, labels=["Low", "Med", "High"])
                binned_cols.append(f"{col}_bin")
        except Exception:
            df[f"{col}_bin"] = pd.cut(df[col], bins=3, labels=["Low", "Med", "High"])
            binned_cols.append(f"{col}_bin")

    cat_cols = [c for c in df.select_dtypes(include=['object', 'category']).columns.tolist() if c not in binary_cols]
    cols_to_dummify = binned_cols + binary_cols + cat_cols
    if not cols_to_dummify:
        raise SystemExit("No columns available to create transactions (after binning). Aborting.")

    dummies = []
    for col in cols_to_dummify:
        s = df[col].astype(str)
        d = pd.get_dummies(s, prefix=col)
        dummies.append(d)
    transactions_ohe = pd.concat(dummies, axis=1)
    # convert to 0/1 ints
    transactions_ohe = transactions_ohe.fillna(0).astype(int)

print("One-hot transaction matrix shape (before pruning):", transactions_ohe.shape)

# Convert to boolean dtype for mlxtend (speeds up computation and avoids deprecation warnings)
transactions_ohe = transactions_ohe.astype(bool)
print("Converted transaction matrix to bool dtype.")

# Prune item columns whose absolute support < min_count (cannot be frequent)
min_count = math.ceil(MIN_SUPPORT * transactions_ohe.shape[0])
col_supports = transactions_ohe.sum(axis=0)
low_support_cols = col_supports[col_supports < min_count].index.tolist()
if len(low_support_cols) > 0:
    transactions_ohe = transactions_ohe.drop(columns=low_support_cols)
    print(f"Dropped {len(low_support_cols)} item columns with support < min_count ({min_count})")
print("Final OHE shape (after low-support pruning):", transactions_ohe.shape)

# Auto-reduce if requested and items > TOP_K_ITEMS
n_items = transactions_ohe.shape[1]
if AUTO_REDUCE and n_items > TOP_K_ITEMS:
    print(f"AUTO_REDUCE engaged: {n_items} items > TOP_K_ITEMS ({TOP_K_ITEMS}). Keeping top-{TOP_K_ITEMS} by support.")
    item_supports = transactions_ohe.sum(axis=0).sort_values(ascending=False)
    top_keep = item_supports.head(TOP_K_ITEMS).index.tolist()
    transactions_ohe = transactions_ohe[top_keep]
    print("Reduced OHE shape:", transactions_ohe.shape)
    n_items = transactions_ohe.shape[1]

# Safety check and warning
if n_items > 150:
    print("\nWARNING: Large number of item columns:", n_items)
    print("Mining may be slow or run out of memory. Consider increasing MIN_SUPPORT or using AUTO_REDUCE.")
    # continue, but user can interrupt if desired

if transactions_ohe.shape[1] == 0:
    raise SystemExit("No item columns remain after pruning. Try lowering MIN_SUPPORT.")

# Run frequent-pattern mining (fpgrowth preferred)
print(f"\nRunning {ALGO} with min_support={MIN_SUPPORT}, max_len={MAX_LEN} ...")
start_time = time.time()
try:
    if ALGO.lower() == "fpgrowth":
        freq_itemsets = fpgrowth(transactions_ohe, min_support=MIN_SUPPORT, use_colnames=True, max_len=MAX_LEN)
    else:
        freq_itemsets = apriori(transactions_ohe, min_support=MIN_SUPPORT, use_colnames=True, max_len=MAX_LEN)
except Exception as e:
    print("Frequent pattern mining failed:", e)
    # fallback attempt with higher support and smaller max_len
    fallback_support = min(max(0.2, MIN_SUPPORT * 2), 0.95)
    print(f"Attempting fallback with higher min_support={fallback_support:.2f} and max_len=2 ...")
    try:
        if ALGO.lower() == "fpgrowth":
            freq_itemsets = fpgrowth(transactions_ohe, min_support=fallback_support, use_colnames=True, max_len=2)
        else:
            freq_itemsets = apriori(transactions_ohe, min_support=fallback_support, use_colnames=True, max_len=2)
        print("Fallback mining succeeded.")
    except Exception as e2:
        raise SystemExit(f"Fallback also failed: {e2}")

elapsed = time.time() - start_time
print(f"Mining completed in {elapsed:.2f}s. Found {len(freq_itemsets)} frequent itemsets.")

if freq_itemsets is None or freq_itemsets.empty:
    print("No frequent itemsets found. Try lowering MIN_SUPPORT or increasing MAX_LEN.")
else:
    freq_itemsets['length'] = freq_itemsets['itemsets'].apply(lambda s: len(s))
    freq_itemsets = freq_itemsets.sort_values(['support','length'], ascending=[False, False]).reset_index(drop=True)
    freq_itemsets['readable'] = freq_itemsets['itemsets'].apply(lambda s: ", ".join(sorted([str(i) for i in s])))
    out_freq = "frequent_itemsets_colab_fast.csv"
    freq_itemsets.to_csv(out_freq, index=False)
    print(f"Saved frequent itemsets to: {out_freq}")
    display(freq_itemsets.head(TOP_N)[['readable','support','length']])

# Generate association rules if itemsets present
if 'freq_itemsets' in locals() and not freq_itemsets.empty:
    print(f"\nGenerating association rules with min_confidence={MIN_CONFIDENCE} ...")
    rules = association_rules(freq_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)
    if rules is None or rules.empty:
        print("No rules generated at this MIN_CONFIDENCE. Consider lowering MIN_CONFIDENCE.")
    else:
        strong_rules = rules[rules['lift'] >= MIN_LIFT].copy()
        strong_rules = strong_rules.sort_values(['confidence','lift','support'], ascending=[False, False, False]).reset_index(drop=True)
        strong_rules['antecedent_readable'] = strong_rules['antecedents'].apply(lambda s: ", ".join(sorted([str(i) for i in s])))
        strong_rules['consequent_readable'] = strong_rules['consequents'].apply(lambda s: ", ".join(sorted([str(i) for i in s])))
        out_rules = "association_rules_colab_fast.csv"
        strong_rules.to_csv(out_rules, index=False)
        print(f"Saved filtered association rules to: {out_rules}")
        if not strong_rules.empty:
            display_cols = ['antecedent_readable','consequent_readable','support','confidence','lift']
            display(strong_rules.head(TOP_N)[display_cols].round(3))
else:
    print("Skipping rule generation because no frequent itemsets were found.")

# Plot rules scatter (support vs confidence colored by lift) if rules exist
if 'rules' in locals() and rules is not None and not rules.empty:
    plt.figure(figsize=(10,6))
    scatter = plt.scatter(rules['support'], rules['confidence'], c=rules['lift'],
                          s=(rules['lift']*40).clip(10,300), cmap='viridis', alpha=0.75, edgecolor='k', linewidth=0.3)
    plt.xlabel("Support")
    plt.ylabel("Confidence")
    plt.title("Association Rules: Confidence vs Support (Color = Lift)")
    cbar = plt.colorbar(scatter)
    cbar.set_label('Lift')
    plt.axhline(MIN_CONFIDENCE, color='r', linestyle='--', linewidth=1, label=f'Min Confidence ({MIN_CONFIDENCE})')
    plt.axvline(MIN_SUPPORT, color='b', linestyle='--', linewidth=1, label=f'Min Support ({MIN_SUPPORT})')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.6)
    plot_file = "apriori_rules_visualization_colab_fast.png"
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"Saved rules scatter plot to: {plot_file}")
    try:
        files.download(out_freq)
    except Exception:
        pass
    try:
        files.download(out_rules)
    except Exception:
        pass
    try:
        files.download(plot_file)
    except Exception:
        pass
    plt.show()
else:
    print("No rule plot created because no rules were generated.")

print("\nDone. Files saved to notebook filesystem and (where possible) downloaded.")

In [ ]:
# Sales Data Warehouse ETL - Based on Star Schema

import pandas as pd
import sqlite3
import numpy as np

# ======================================================
# Step 1: EXTRACT - Raw Sales Data
# ======================================================

# Time Dimension
dim_time = pd.DataFrame({
    'time_id': [1, 2, 3, 4, 5],
    'date': pd.to_datetime(['2024-01-15', '2024-02-20', '2024-03-10', '2024-04-05', '2024-05-12']),
    'day': [15, 20, 10, 5, 12],
    'month': [1, 2, 3, 4, 5],
    'year': [2024, 2024, 2024, 2024, 2024]
})

# Sales Channel Dimension
dim_sales_channel = pd.DataFrame({
    'channel_id': [1, 2],
    'channel_name': ['online', 'in-store'],
    'channel_type': ['digital', 'physical']
})

# Customer Dimension
dim_customer = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5],
    'name': ['John Smith', 'Sarah Johnson', 'Mike Davis', 'Emily Wilson', 'David Brown'],
    'age': [28, 35, 42, 31, 29],
    'gender': ['M', 'F', 'M', 'F', 'M'],
    'segment': ['Premium', 'Standard', 'Premium', 'Standard', 'Premium']
})

# Region Dimension
dim_region = pd.DataFrame({
    'region_id': [1, 2, 3],
    'name': ['North America', 'Europe', 'Asia'],
    'country': ['USA', 'Germany', 'Japan'],
    'state': ['California', 'Bavaria', 'Tokyo'],
    'city': ['Los Angeles', 'Munich', 'Tokyo']
})

# Product Dimension
dim_product = pd.DataFrame({
    'product_id': [1, 2, 3, 4],
    'name': ['Laptop Pro', 'Smartphone X', 'Tablet Plus', 'Headphones'],
    'category': ['Electronics', 'Electronics', 'Electronics', 'Accessories'],
    'price': [1200.00, 800.00, 600.00, 150.00],
    'supplier': ['TechCorp', 'MobileTech', 'TabletInc', 'AudioMax']
})

# Sales Fact Table (Raw Data)
fact_sales_raw = pd.DataFrame({
    'sale_id': [101, 102, 103, 104, 105, 106, 107, 108],
    'time_id': [1, 2, 3, 4, 5, 1, 2, 3],
    'channel_id': [1, 2, 1, 2, 1, 2, 1, 2],
    'customer_id': [1, 2, 3, 4, 5, 2, 1, 3],
    'region_id': [1, 2, 3, 1, 2, 1, 3, 2],
    'product_id': [1, 2, 3, 4, 1, 2, 3, 4],
    'no_of_transactions': [1, 2, 1, 3, 1, 2, 1, 1],
    'discount_offered': [100.00, 50.00, 0.00, 20.00, 150.00, 75.00, 30.00, 10.00]
})

# ======================================================
# Step 2: TRANSFORM - Calculate Derived Fields
# ======================================================

# Add product price to sales data for total sales calculation
fact_sales = fact_sales_raw.merge(dim_product[['product_id', 'price']], on='product_id')

# Calculate total sales (price * quantity - discount)
fact_sales['total_sales'] = (fact_sales['price'] * fact_sales['no_of_transactions']) - fact_sales['discount_offered']

# Final fact table with required columns
fact_sales_final = fact_sales[['sale_id', 'time_id', 'channel_id', 'customer_id', 'region_id', 'product_id',
                               'total_sales', 'no_of_transactions', 'discount_offered']]

print("Sample Data Preview:")
print("\n📊 Time Dimension:")
print(dim_time.head())
print("\n📊 Sales Channel Dimension:")
print(dim_sales_channel)
print("\n📊 Customer Dimension:")
print(dim_customer.head())
print("\n📊 Sales Fact Table:")
print(fact_sales_final.head())

# ======================================================
# Step 3: LOAD - Store in SQLite Data Warehouse
# ======================================================

conn = sqlite3.connect('sales_dw.db')
cursor = conn.cursor()

# Save each dimension and fact table
dim_time.to_sql('dim_time', conn, if_exists='replace', index=False)
dim_sales_channel.to_sql('dim_sales_channel', conn, if_exists='replace', index=False)
dim_customer.to_sql('dim_customer', conn, if_exists='replace', index=False)
dim_region.to_sql('dim_region', conn, if_exists='replace', index=False)
dim_product.to_sql('dim_product', conn, if_exists='replace', index=False)
fact_sales_final.to_sql('fact_sales', conn, if_exists='replace', index=False)

print("\n✅ Sales ETL completed successfully. Data loaded into 'sales_dw.db'.")
print(f"📁 Database contains {len(fact_sales_final)} sales transactions")
print(f"📁 Spanning {len(dim_time)} time periods, {len(dim_customer)} customers, and {len(dim_product)} products")
conn.close()

In [ ]:
# ======================================================
# Step 4: VIEW SALES DATABASE CONTENTS
# ======================================================

import os
import pandas as pd
import sqlite3

# Check if database file exists and show its location
db_path = 'sales_dw.db'
if os.path.exists(db_path):
    print(f"✅ Sales Database found at: {os.path.abspath(db_path)}")
    print(f"📁 File size: {os.path.getsize(db_path)} bytes")
else:
    print("❌ Sales Database not found. Run the ETL code first!")

# Connect and display all tables
conn = sqlite3.connect(db_path)

# Show all tables in the database
print("\n" + "="*60)
print("📊 TABLES IN SALES DATA WAREHOUSE:")
print("="*60)
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)
print(tables.to_string(index=False))

# Display contents of each table
table_names = ['dim_time', 'dim_sales_channel', 'dim_customer', 'dim_region', 'dim_product', 'fact_sales']

for table in table_names:
    try:
        print(f"\n" + "="*60)
        print(f"📋 CONTENTS OF {table.upper()}:")
        print("="*60)
        df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
        print(df.to_string(index=False))
        print(f"\n📊 Statistics: {len(df)} rows, {len(df.columns)} columns")

        # Show some key stats for fact table
        if table == 'fact_sales':
            total_sales = df['total_sales'].sum()
            avg_sales = df['total_sales'].mean()
            print(f"💰 Total Sales: ${total_sales:,.2f}")
            print(f"📈 Average Sale: ${avg_sales:,.2f}")

    except Exception as e:
        print(f"⚠️ Could not read {table}: {e}")

conn.close()
print("\n✅ Sales Database view completed!")

In [ ]:
# Sales Data Warehouse OLAP Analysis

import pandas as pd
import sqlite3

# Connect to Sales Data Warehouse
conn = sqlite3.connect('sales_dw.db')

# Helper function to display query results neatly
def show_query(title, query):
    print(f"\n{'='*60}")
    print(f"📊 {title}")
    print('='*60)
    df = pd.read_sql_query(query, conn)
    print(df.to_string(index=False))
    if 'total_sales' in df.columns or 'Total_Sales' in df.columns:
        sales_col = 'total_sales' if 'total_sales' in df.columns else 'Total_Sales'
        print(f"\n💰 Total Sales: ${df[sales_col].sum():,.2f}")

# ======================================================
# SALES OLAP OPERATIONS
# ======================================================

# (a) ROLL-UP: Aggregate sales by region
query_rollup = """
SELECT r.name as Region, SUM(s.total_sales) AS Total_Sales
FROM fact_sales s
JOIN dim_region r ON s.region_id = r.region_id
GROUP BY r.name
ORDER BY Total_Sales DESC;
"""
show_query("ROLL-UP: Sales by Region", query_rollup)

# (b) DRILL-DOWN: Region → Country → City
query_drilldown = """
SELECT r.name as Region, r.country, r.city, SUM(s.total_sales) AS City_Sales
FROM fact_sales s
JOIN dim_region r ON s.region_id = r.region_id
GROUP BY r.name, r.country, r.city
ORDER BY City_Sales DESC;
"""
show_query("DRILL-DOWN: Sales by Region → Country → City", query_drilldown)

# (c) SLICE: Filter by sales channel (online only)
query_slice = """
SELECT p.name as Product, SUM(s.total_sales) AS Online_Sales,
       SUM(s.no_of_transactions) as Transactions
FROM fact_sales s
JOIN dim_product p ON s.product_id = p.product_id
JOIN dim_sales_channel c ON s.channel_id = c.channel_id
WHERE c.channel_name = 'online'
GROUP BY p.name
ORDER BY Online_Sales DESC;
"""
show_query("SLICE: Online Sales by Product", query_slice)

# (d) DICE: Filter by Premium customers AND Electronics category
query_dice = """
SELECT r.name as Region, p.name as Product, SUM(s.total_sales) AS Premium_Electronics_Sales
FROM fact_sales s
JOIN dim_customer c ON s.customer_id = c.customer_id
JOIN dim_product p ON s.product_id = p.product_id
JOIN dim_region r ON s.region_id = r.region_id
WHERE c.segment = 'Premium' AND p.category = 'Electronics'
GROUP BY r.name, p.name
ORDER BY Premium_Electronics_Sales DESC;
"""
show_query("DICE: Premium Customer Electronics Sales by Region", query_dice)

# (e) PIVOT: Sales Channel vs Product Category
query_pivot = """
SELECT p.category as Product_Category,
  SUM(CASE WHEN c.channel_name = 'online' THEN s.total_sales ELSE 0 END) AS Online_Sales,
  SUM(CASE WHEN c.channel_name = 'in-store' THEN s.total_sales ELSE 0 END) AS InStore_Sales,
  SUM(s.total_sales) AS Total_Sales
FROM fact_sales s
JOIN dim_product p ON s.product_id = p.product_id
JOIN dim_sales_channel c ON s.channel_id = c.channel_id
GROUP BY p.category
ORDER BY Total_Sales DESC;
"""
show_query("PIVOT: Sales Channel vs Product Category", query_pivot)

# (f) TIME ANALYSIS: Monthly sales trend
query_time = """
SELECT t.month, t.year, SUM(s.total_sales) AS Monthly_Sales,
       COUNT(s.sale_id) AS Total_Transactions,
       AVG(s.total_sales) AS Avg_Sale_Value
FROM fact_sales s
JOIN dim_time t ON s.time_id = t.time_id
GROUP BY t.year, t.month
ORDER BY t.year, t.month;
"""
show_query("TIME ANALYSIS: Monthly Sales Trend", query_time)

# (g) CUSTOMER ANALYSIS: Top customers by sales
query_customers = """
SELECT c.name as Customer, c.segment, SUM(s.total_sales) AS Customer_Sales,
       COUNT(s.sale_id) as Purchases, AVG(s.total_sales) as Avg_Purchase
FROM fact_sales s
JOIN dim_customer c ON s.customer_id = c.customer_id
GROUP BY c.customer_id, c.name, c.segment
ORDER BY Customer_Sales DESC;
"""
show_query("CUSTOMER ANALYSIS: Top Customers", query_customers)

# ======================================================
# CLOSE CONNECTION
# ======================================================
conn.close()
print(f"\n{'='*60}")
print("✅ Sales OLAP analysis completed successfully!")
print("📊 Analysis included: Roll-up, Drill-down, Slice, Dice, Pivot, Time & Customer Analysis")
print("="*60)